In [1]:
%load_ext autoreload

In [3]:
%autoreload 2

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import getpass
from IPython.display import display
import json
import nltk
import numpy as np
import pandas as pd
import pkg_resources
import os
import random
import re
import seaborn as sns
import sklearn.metrics as metrics

import tensorflow as tf
from tensorflow.python.lib.io import file_io

In [159]:
from utils_export.dataset import Dataset, Model
from utils_export import utils_cloudml
from utils_export import utils_tfrecords

In [6]:
# Faster to access GCS file:
# https://github.com/tensorflow/tensorflow/issues/15530
os.environ['GCS_READ_CACHE_MAX_SIZE_MB'] = '0'

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/google/home/msushkov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [117]:
def tokenizer(text, lowercase=True):
  """Converts text to a list of words.

  Args:
    text: piece of text to tokenize (string).
    lowercase: whether to include lowercasing in preprocessing (bool).

  Returns:
    A list of strings (words).
  """
  words = nltk.word_tokenize(text.decode('utf-8'))
  if lowercase:
    words = [w.lower() for w in words]
  return words

In [208]:
def make_test_input_fn(dataset_path,
                       model_text_feature,
                       dataset_text_feature,
                       data_label,
                       tokenizer_fn,
                       label_data_type=tf.float32,
                       max_n_examples=None,
                       random_filter_keep_rate=1.0):
    """Returns a test input function.
    
    Args:
      dataset_path (str): Path to dataset.
      model_text_feature (str): The feature column corresponding to the
        text input the model expects.
      dataset_text_feature (str): The name of the text feature of the dataset.
      data_label (str): The output label for the dataset.
      tokenizer_fn: Tokenizer function (str -> list).
      max_n_examples (int): How many examples to evaluate on.
      random_filter_keep_rate (float): Filter out test examples with this probability.

    Returns:
      Test input function.
    """
    decoding_input_features = {
      dataset_text_feature: tf.FixedLenFeature([], dtype=tf.string),
      data_label: tf.FixedLenFeature([], dtype=label_data_type)
    }

    def test_input_fn(max_n_examples=max_n_examples,
                      random_filter_keep_rate=random_filter_keep_rate):
        """Test input function.
        
        Args:
          max_n_examples (int): How many examples to evaluate on.
          random_filter_keep_rate (float): Filter out test examples with this probability.
          
        Returns:
          DataFrame with the results.
        """
        res = utils_tfrecords.decode_tf_records_to_pandas(
            decoding_input_features,
            dataset_path,
            max_n_examples,
            random_filter_keep_rate)
        if not tokenizer_fn:
            tok = lambda x: [x]
            res[model_text_feature] = list(map(tok, res[dataset_text_feature]))
        else:
            res[model_text_feature] = list(map(tokenizer_fn, res[dataset_text_feature]))
        res = res.rename(columns={ data_label: 'label' })
        res['label'] = list(map(lambda x: bool(round(x)), list(res['label'])))
        final = res.copy(deep=True)
        return final

    return test_input_fn

In [134]:
def print_results(results_df, model_names):
    """Print the classification results.
    
    Args:
      results_df: DataFrame with the results.
      model_names: List of strings representing the models for which we have results.
    """
    labels = results_df['label']
    for _model in model_names:
        print(_model)
        model_preds = results_df[_model]
        fpr, tpr, thresholds = metrics.roc_curve(labels, model_preds)
        roc_auc = metrics.auc(fpr, tpr)
        recalls, precisions, thr = metrics.precision_recall_curve(labels, model_preds)
        pr_auc = metrics.auc(precisions, recalls)
        print('\tROC AUC: {}'.format(roc_auc))
        print('\tPR AUC: {}'.format(pr_auc))

In [121]:
PROJECT_NAME = 'conversationai-models'
SENTENCE_KEY = 'comment_key' #Input key

# Pattern for path of tf_records
OUTPUT_DIR_BASE = os.path.join(
    'gs://conversationai-models',
    getpass.getuser(),
    'tfrecords')

## Evaluate models on Civil Comments dataset

In [175]:
LABEL_NAME_PREDICTION_MODEL = 'toxicity/logistic'
DATASET = 'gs://conversationai-models/resources/civil_comments_data/train_eval_test/test-*.tfrecord'
DATA_LABEL = 'toxicity'
DATASET_TEXT_FEATURE='comment_text'

# Pattern for path of tf_records
OUTPUT_DIR = os.path.join(OUTPUT_DIR_BASE, 'civil_comments_test')

### CNN, GRU Attention Models

In [140]:
MODEL_TEXT_FEATURE = 'tokens'
MODEL_NAMES = [
    'tf_cnn_civil_comments_glove:v_20190219_185541',
    'tf_gru_attention_civil_comments_glove:v_20190219_185619',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, tokenizer)

In [141]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(10000000)

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:Loaded 200000 lines.
INFO:tensorflow:End of file.


In [142]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:TF-Records already exist - overwriting them.
INFO:tensorflow:Preparing train data: 0/242144
INFO:tensorflow:Preparing train data: 10000/242144
INFO:tensorflow:Preparing train data: 20000/242144
INFO:tensorflow:Preparing train data: 30000/242144
INFO:tensorflow:Preparing train data: 40000/242144
INFO:tensorflow:Preparing train data: 50000/242144
INFO:tensorflow:Preparing train data: 60000/242144
INFO:tensorflow:Preparing train data: 70000/242144
INFO:tensorflow:Preparing train data: 80000/242144
INFO:tensorflow:Preparing train data: 90000/242144
INFO:tensorflow:Preparing train data: 100000/242144
INFO:tensorflow:Preparing train data: 110000/242144
INFO:tensorflow:Preparing train data: 120000/242144
INFO:tensorflow:Preparing train data: 130000/242144
INFO:tensorflow:Preparing train data: 140000/242144
INFO:tensorflow:Preparing train data: 150000/242144
INFO:tensorflow:Preparin

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Prediction job completed.


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Prediction job completed.


In [143]:
civil_comments_test_df = test_dataset.show_data()

In [144]:
print_results(civil_comments_test_df, MODEL_NAMES)

tf_cnn_civil_comments_glove:v_20190219_185541
	ROC AUC: 0.9573435242534393
	PR AUC: 0.6729934425219886
tf_gru_attention_civil_comments_glove:v_20190219_185619
	ROC AUC: 0.9649161132104584
	PR AUC: 0.7486011745102973


### TF-Hub Model

In [189]:
MODEL_TEXT_FEATURE = 'text'
MODEL_NAMES = [
    'tf_hub_classifier_civil_comments:v20190322_142141_21201_1553344552',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, None)

In [190]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(10000000)

INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:Loaded 200000 lines.
INFO:tensorflow:End of file.


In [192]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:TF-Records already exist - overwriting them.
INFO:tensorflow:Preparing train data: 0/242144
INFO:tensorflow:Preparing train data: 10000/242144
INFO:tensorflow:Preparing train data: 20000/242144
INFO:tensorflow:Preparing train data: 30000/242144
INFO:tensorflow:Preparing train data: 40000/242144
INFO:tensorflow:Preparing train data: 50000/242144
INFO:tensorflow:Preparing train data: 60000/242144
INFO:tensorflow:Preparing train data: 70000/242144
INFO:tensorflow:Preparing train data: 80000/242144
INFO:tensorflow:Preparing train data: 90000/242144
INFO:tensorflow:Preparing train data: 100000/242144
INFO:tensorflow:Preparing train data: 110000/242144
INFO:tensorflow:Preparing train data: 120000/242144
INFO:tensorflow:Preparing train data: 130000/242144
INFO:tensorflow:Preparing train data: 140000/242144
INFO:tensorflow:Preparing train data: 150000/242144
INFO:tensorflow:Preparin

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 25
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 27
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 30
INFO:tensorflow:

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 269
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 271
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 274
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 276
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 279
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 281
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 284
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 286
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 289
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 291
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 294
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 296
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 299

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 538
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 540
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 543
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 545
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 548
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 550
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 553
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 555
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 558
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 560
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 563
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 565
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 568

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 807
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 809
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 812
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 814
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 817
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 819
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 822
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 824
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 827
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 829
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 832
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 834
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 837

In [193]:
civil_comments_hub_df = test_dataset.show_data()

In [194]:
print_results(civil_comments_hub_df, MODEL_NAMES)

tf_hub_classifier_civil_comments:v20190322_142141_21201_1553344552
	ROC AUC: 0.9595451744696132
	PR AUC: 0.7429338592289392


## Evaluate models on Toxicity dataset

In [195]:
LABEL_NAME_PREDICTION_MODEL = 'frac_neg/logistic'
DATASET = 'gs://conversationai-models/resources/toxicity_data/toxicity_q42017_test.tfrecord'
DATA_LABEL = 'frac_neg'
DATASET_TEXT_FEATURE='comment_text'

# Pattern for path of tf_records
OUTPUT_DIR = os.path.join(OUTPUT_DIR_BASE, 'toxicity_test')

### CNN, GRU Attention Models

In [162]:
MODEL_TEXT_FEATURE = 'tokens'
MODEL_NAMES = [
    'tf_cnn_toxicity_glove:v_20190219_185532',
    'tf_gru_attention_toxicity_glove:v_20190219_185516',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, tokenizer)

In [163]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(10000000)

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:End of file.


In [148]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:Preparing train data: 0/187681
INFO:tensorflow:Preparing train data: 10000/187681
INFO:tensorflow:Preparing train data: 20000/187681
INFO:tensorflow:Preparing train data: 30000/187681
INFO:tensorflow:Preparing train data: 40000/187681
INFO:tensorflow:Preparing train data: 50000/187681
INFO:tensorflow:Preparing train data: 60000/187681
INFO:tensorflow:Preparing train data: 70000/187681
INFO:tensorflow:Preparing train data: 80000/187681
INFO:tensorflow:Preparing train data: 90000/187681
INFO:tensorflow:Preparing train data: 100000/187681
INFO:tensorflow:Preparing train data: 110000/187681
INFO:tensorflow:Preparing train data: 120000/187681
INFO:tensorflow:Preparing train data: 130000/187681
INFO:tensorflow:Preparing train data: 140000/187681
INFO:tensorflow:Preparing train data: 150000/187681
INFO:tensorflow:Preparing train data: 160000/187681
INFO:tensorflow:Preparing train d

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 25
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 27
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 30
INFO:tensorflow:

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Prediction job completed.


In [149]:
toxicity_test_df1 = test_dataset.show_data()

In [150]:
print_results(toxicity_test_df1, MODEL_NAMES)

tf_cnn_toxicity_glove:v_20190219_185532
	ROC AUC: 0.951760553925346
	PR AUC: 0.8740274773143215
tf_gru_attention_toxicity_glove:v_20190219_185516
	ROC AUC: 0.9543916575133977
	PR AUC: 0.8814208812923074


### TF-Hub Model

In [196]:
MODEL_TEXT_FEATURE = 'text'
MODEL_NAMES = [
    'tf_hub_classifier_toxicity:v20190322_142740_24239_1553555427',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, None)

In [197]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(10000000)

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:End of file.


In [199]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:TF-Records already exist - overwriting them.
INFO:tensorflow:Preparing train data: 0/187681
INFO:tensorflow:Preparing train data: 10000/187681
INFO:tensorflow:Preparing train data: 20000/187681
INFO:tensorflow:Preparing train data: 30000/187681
INFO:tensorflow:Preparing train data: 40000/187681
INFO:tensorflow:Preparing train data: 50000/187681
INFO:tensorflow:Preparing train data: 60000/187681
INFO:tensorflow:Preparing train data: 70000/187681
INFO:tensorflow:Preparing train data: 80000/187681
INFO:tensorflow:Preparing train data: 90000/187681
INFO:tensorflow:Preparing train data: 100000/187681
INFO:tensorflow:Preparing train data: 110000/187681
INFO:tensorflow:Preparing train data: 120000/187681
INFO:tensorflow:Preparing train data: 130000/187681
INFO:tensorflow:Preparing train data: 140000/187681
INFO:tensorflow:Preparing train data: 150000/187681
INFO:tensorflow:Preparin

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 25
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 27
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 30
INFO:tensorflow:

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 269
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 271
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 274
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 276
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 279
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 281
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 284
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 286
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 289
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 291
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 294
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 296
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 299

In [200]:
toxicity_test_df2 = test_dataset.show_data()

In [201]:
print_results(toxicity_test_df2, MODEL_NAMES)

tf_hub_classifier_toxicity:v20190322_142740_24239_1553555427
	ROC AUC: 0.9270843170934745
	PR AUC: 0.8155815559085313


## Evaluate models on Many Communities data

In [314]:
LABEL_NAME_PREDICTION_MODEL = 'removed/logistic'
DATASET = 'gs://conversationai-models/resources/transfer_learning_data/many_communities/20181105_answers_all_columns_nthain.tfrecord'
#DATASET = 'gs://conversationai-models/resources/transfer_learning_data/many_communities/20181105_answers_all_columns_nthain_1k.tfrecord'
DATA_LABEL = 'removed'
DATASET_TEXT_FEATURE='comment_text'

# Pattern for path of tf_records
OUTPUT_DIR = os.path.join(OUTPUT_DIR_BASE, 'many_communities_test')

### CNN, GRU Attention Models

In [315]:
MODEL_TEXT_FEATURE = 'tokens'
MODEL_NAMES = [
    'tf_cnn_many_communities_glove:v_20190219_185551',
    #'tf_gru_attention_many_communities:v20190322_142800_507893_1556085643',
    'tf_gru_attention_many_communities:v20190315_161037_23271_1555129264',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, tokenizer, label_data_type=tf.int64)

In [316]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(100000000)

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:Loaded 200000 lines.
INFO:tensorflow:Loaded 300000 lines.
INFO:tensorflow:Loaded 400000 lines.
INFO:tensorflow:Loaded 500000 lines.
INFO:tensorflow:Loaded 600000 lines.
INFO:tensorflow:Loaded 700000 lines.
INFO:tensorflow:Loaded 800000 lines.
INFO:tensorflow:Loaded 900000 lines.
INFO:tensorflow:Loaded 1000000 lines.
INFO:tensorflow:Loaded 1100000 lines.
INFO:tensorflow:Loaded 1200000 lines.
INFO:tensorflow:Loaded 1300000 lines.
INFO:tensorflow:Loaded 1400000 lines.
INFO:tensorflow:Loaded 1500000 lines.
INFO:tensorflow:Loaded 1600000 lines.
INFO:tensorflow:Loaded 1700000 lines.
INFO:tensorflow:Loaded 1800000 lines.
INFO:tensorflow:Loaded 1900000 lines.
INFO:tensorflow:Loaded 2000000 lines.
INFO:tensorflow:Loaded 2100000 lines.
INFO:tensorflow:Loaded 2200000 lines.
INFO:tensorflow:Loaded 2300000 lines.
INFO:tensorflow:Loaded 2400000 lines.
INFO:tensorflow:Loaded 2500000 l

In [317]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:TF-Records already exist - overwriting them.
INFO:tensorflow:Preparing train data: 0/5731960
INFO:tensorflow:Preparing train data: 10000/5731960
INFO:tensorflow:Preparing train data: 20000/5731960
INFO:tensorflow:Preparing train data: 30000/5731960
INFO:tensorflow:Preparing train data: 40000/5731960
INFO:tensorflow:Preparing train data: 50000/5731960
INFO:tensorflow:Preparing train data: 60000/5731960
INFO:tensorflow:Preparing train data: 70000/5731960
INFO:tensorflow:Preparing train data: 80000/5731960
INFO:tensorflow:Preparing train data: 90000/5731960
INFO:tensorflow:Preparing train data: 100000/5731960
INFO:tensorflow:Preparing train data: 110000/5731960
INFO:tensorflow:Preparing train data: 120000/5731960
INFO:tensorflow:Preparing train data: 130000/5731960
INFO:tensorflow:Preparing train data: 140000/5731960
INFO:tensorflow:Preparing train data: 150000/5731960
INFO:ten

INFO:tensorflow:Preparing train data: 1510000/5731960
INFO:tensorflow:Preparing train data: 1520000/5731960
INFO:tensorflow:Preparing train data: 1530000/5731960
INFO:tensorflow:Preparing train data: 1540000/5731960
INFO:tensorflow:Preparing train data: 1550000/5731960
INFO:tensorflow:Preparing train data: 1560000/5731960
INFO:tensorflow:Preparing train data: 1570000/5731960
INFO:tensorflow:Preparing train data: 1580000/5731960
INFO:tensorflow:Preparing train data: 1590000/5731960
INFO:tensorflow:Preparing train data: 1600000/5731960
INFO:tensorflow:Preparing train data: 1610000/5731960
INFO:tensorflow:Preparing train data: 1620000/5731960
INFO:tensorflow:Preparing train data: 1630000/5731960
INFO:tensorflow:Preparing train data: 1640000/5731960
INFO:tensorflow:Preparing train data: 1650000/5731960
INFO:tensorflow:Preparing train data: 1660000/5731960
INFO:tensorflow:Preparing train data: 1670000/5731960
INFO:tensorflow:Preparing train data: 1680000/5731960
INFO:tensorflow:Preparing tr

INFO:tensorflow:Preparing train data: 3030000/5731960
INFO:tensorflow:Preparing train data: 3040000/5731960
INFO:tensorflow:Preparing train data: 3050000/5731960
INFO:tensorflow:Preparing train data: 3060000/5731960
INFO:tensorflow:Preparing train data: 3070000/5731960
INFO:tensorflow:Preparing train data: 3080000/5731960
INFO:tensorflow:Preparing train data: 3090000/5731960
INFO:tensorflow:Preparing train data: 3100000/5731960
INFO:tensorflow:Preparing train data: 3110000/5731960
INFO:tensorflow:Preparing train data: 3120000/5731960
INFO:tensorflow:Preparing train data: 3130000/5731960
INFO:tensorflow:Preparing train data: 3140000/5731960
INFO:tensorflow:Preparing train data: 3150000/5731960
INFO:tensorflow:Preparing train data: 3160000/5731960
INFO:tensorflow:Preparing train data: 3170000/5731960
INFO:tensorflow:Preparing train data: 3180000/5731960
INFO:tensorflow:Preparing train data: 3190000/5731960
INFO:tensorflow:Preparing train data: 3200000/5731960
INFO:tensorflow:Preparing tr

INFO:tensorflow:Preparing train data: 4550000/5731960
INFO:tensorflow:Preparing train data: 4560000/5731960
INFO:tensorflow:Preparing train data: 4570000/5731960
INFO:tensorflow:Preparing train data: 4580000/5731960
INFO:tensorflow:Preparing train data: 4590000/5731960
INFO:tensorflow:Preparing train data: 4600000/5731960
INFO:tensorflow:Preparing train data: 4610000/5731960
INFO:tensorflow:Preparing train data: 4620000/5731960
INFO:tensorflow:Preparing train data: 4630000/5731960
INFO:tensorflow:Preparing train data: 4640000/5731960
INFO:tensorflow:Preparing train data: 4650000/5731960
INFO:tensorflow:Preparing train data: 4660000/5731960
INFO:tensorflow:Preparing train data: 4670000/5731960
INFO:tensorflow:Preparing train data: 4680000/5731960
INFO:tensorflow:Preparing train data: 4690000/5731960
INFO:tensorflow:Preparing train data: 4700000/5731960
INFO:tensorflow:Preparing train data: 4710000/5731960
INFO:tensorflow:Preparing train data: 4720000/5731960
INFO:tensorflow:Preparing tr

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


...

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 25
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 27
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 30
INFO:tensorflow:

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 269
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 271
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 274
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 276
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 279
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 281
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 284
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 286
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 289
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 291
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 294
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 296
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 299

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 538
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 540
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 543
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 545
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 548
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 550
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 553
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 555
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 558
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 560
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 563
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 565
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 568

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Prediction job completed.


In [318]:
many_communities_test_df = test_dataset.show_data()

In [295]:
np.sum(np.isnan(many_communities_test_df["tf_cnn_many_communities_glove:v_20190219_185551"].values))

0

In [319]:
print_results(many_communities_test_df, MODEL_NAMES)

tf_cnn_many_communities_glove:v_20190219_185551
	ROC AUC: 0.7476941464055139
	PR AUC: 0.07604839414024091
tf_gru_attention_many_communities:v20190315_161037_23271_1555129264
	ROC AUC: 0.7215269560475308
	PR AUC: 0.06656538517176142


### TF-Hub Model

In [320]:
MODEL_TEXT_FEATURE = 'text'
MODEL_NAMES = [
    'tf_hub_classifier_many_communities:v20190219_185602_316000_1553563221',
]

model_input_spec = {
    MODEL_TEXT_FEATURE: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
}

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

test_input_fn = make_test_input_fn(
    DATASET, MODEL_TEXT_FEATURE, DATASET_TEXT_FEATURE,
    DATA_LABEL, None, label_data_type=tf.int64)

In [321]:
# Need to set seed before loading data to be able to reload same data in the future
random.seed(2018)

test_dataset = Dataset(test_input_fn, OUTPUT_DIR)
test_dataset.load_data(10000000)

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:input_fn is compatible with the `Dataset` class.
INFO:tensorflow:Loaded 100000 lines.
INFO:tensorflow:Loaded 200000 lines.
INFO:tensorflow:Loaded 300000 lines.
INFO:tensorflow:Loaded 400000 lines.
INFO:tensorflow:Loaded 500000 lines.
INFO:tensorflow:Loaded 600000 lines.
INFO:tensorflow:Loaded 700000 lines.
INFO:tensorflow:Loaded 800000 lines.
INFO:tensorflow:Loaded 900000 lines.
INFO:tensorflow:Loaded 1000000 lines.
INFO:tensorflow:Loaded 1100000 lines.
INFO:tensorflow:Loaded 1200000 lines.
INFO:tensorflow:Loaded 1300000 lines.
INFO:tensorflow:Loaded 1400000 lines.
INFO:tensorflow:Loaded 1500000 lines.
INFO:tensorflow:Loaded 1600000 lines.
INFO:tensorflow:Loaded 1700000 lines.
INFO:tensorflow:Loaded 1800000 lines.
INFO:tensorflow:Loaded 1900000 lines.
INFO:tensorflow:Loaded 2000000 lines.
INFO:tensorflow:Loaded 2100000 lines.
INFO:tensorflow:Loaded 2200000 lines.
INFO:tensorflow:Loaded 2300000 lines.
INFO:tensorflow:Loaded 2400000 lines.
INFO:tensorflow:Loaded 2500000 l

In [ ]:
# Set recompute_predictions=False to save time if predictions are available.
test_dataset.add_model_prediction_to_data(model, recompute_predictions=True)

INFO:tensorflow:Model is compatible with the `Dataset` instance.
INFO:tensorflow:Doing batch 0/0
INFO:tensorflow:TF-Records already exist - overwriting them.
INFO:tensorflow:Preparing train data: 0/5731960
INFO:tensorflow:Preparing train data: 10000/5731960
INFO:tensorflow:Preparing train data: 20000/5731960
INFO:tensorflow:Preparing train data: 30000/5731960
INFO:tensorflow:Preparing train data: 40000/5731960
INFO:tensorflow:Preparing train data: 50000/5731960
INFO:tensorflow:Preparing train data: 60000/5731960
INFO:tensorflow:Preparing train data: 70000/5731960
INFO:tensorflow:Preparing train data: 80000/5731960
INFO:tensorflow:Preparing train data: 90000/5731960
INFO:tensorflow:Preparing train data: 100000/5731960
INFO:tensorflow:Preparing train data: 110000/5731960
INFO:tensorflow:Preparing train data: 120000/5731960
INFO:tensorflow:Preparing train data: 130000/5731960
INFO:tensorflow:Preparing train data: 140000/5731960
INFO:tensorflow:Preparing train data: 150000/5731960
INFO:ten

INFO:tensorflow:Preparing train data: 1510000/5731960
INFO:tensorflow:Preparing train data: 1520000/5731960
INFO:tensorflow:Preparing train data: 1530000/5731960
INFO:tensorflow:Preparing train data: 1540000/5731960
INFO:tensorflow:Preparing train data: 1550000/5731960
INFO:tensorflow:Preparing train data: 1560000/5731960
INFO:tensorflow:Preparing train data: 1570000/5731960
INFO:tensorflow:Preparing train data: 1580000/5731960
INFO:tensorflow:Preparing train data: 1590000/5731960
INFO:tensorflow:Preparing train data: 1600000/5731960
INFO:tensorflow:Preparing train data: 1610000/5731960
INFO:tensorflow:Preparing train data: 1620000/5731960
INFO:tensorflow:Preparing train data: 1630000/5731960
INFO:tensorflow:Preparing train data: 1640000/5731960
INFO:tensorflow:Preparing train data: 1650000/5731960
INFO:tensorflow:Preparing train data: 1660000/5731960
INFO:tensorflow:Preparing train data: 1670000/5731960
INFO:tensorflow:Preparing train data: 1680000/5731960
INFO:tensorflow:Preparing tr

INFO:tensorflow:Preparing train data: 3030000/5731960
INFO:tensorflow:Preparing train data: 3040000/5731960
INFO:tensorflow:Preparing train data: 3050000/5731960
INFO:tensorflow:Preparing train data: 3060000/5731960
INFO:tensorflow:Preparing train data: 3070000/5731960
INFO:tensorflow:Preparing train data: 3080000/5731960
INFO:tensorflow:Preparing train data: 3090000/5731960
INFO:tensorflow:Preparing train data: 3100000/5731960
INFO:tensorflow:Preparing train data: 3110000/5731960
INFO:tensorflow:Preparing train data: 3120000/5731960
INFO:tensorflow:Preparing train data: 3130000/5731960
INFO:tensorflow:Preparing train data: 3140000/5731960
INFO:tensorflow:Preparing train data: 3150000/5731960
INFO:tensorflow:Preparing train data: 3160000/5731960
INFO:tensorflow:Preparing train data: 3170000/5731960
INFO:tensorflow:Preparing train data: 3180000/5731960
INFO:tensorflow:Preparing train data: 3190000/5731960
INFO:tensorflow:Preparing train data: 3200000/5731960
INFO:tensorflow:Preparing tr

INFO:tensorflow:Preparing train data: 4550000/5731960
INFO:tensorflow:Preparing train data: 4560000/5731960
INFO:tensorflow:Preparing train data: 4570000/5731960
INFO:tensorflow:Preparing train data: 4580000/5731960
INFO:tensorflow:Preparing train data: 4590000/5731960
INFO:tensorflow:Preparing train data: 4600000/5731960
INFO:tensorflow:Preparing train data: 4610000/5731960
INFO:tensorflow:Preparing train data: 4620000/5731960
INFO:tensorflow:Preparing train data: 4630000/5731960
INFO:tensorflow:Preparing train data: 4640000/5731960
INFO:tensorflow:Preparing train data: 4650000/5731960
INFO:tensorflow:Preparing train data: 4660000/5731960
INFO:tensorflow:Preparing train data: 4670000/5731960
INFO:tensorflow:Preparing train data: 4680000/5731960
INFO:tensorflow:Preparing train data: 4690000/5731960
INFO:tensorflow:Preparing train data: 4700000/5731960
INFO:tensorflow:Preparing train data: 4710000/5731960
INFO:tensorflow:Preparing train data: 4720000/5731960
INFO:tensorflow:Preparing tr

/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:state : QUEUED


/usr/local/google/home/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 0
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 2
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 5
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 7
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 10
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 12
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 15
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 17
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 20
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 22
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 25
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 27
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 30
INFO:tensorflow:

INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 269
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 271
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 274
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 276
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 279
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 281
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 284
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 286
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 289
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 291
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 294
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 296
INFO:tensorflow:Waiting for prediction job to complete. Minutes elapsed: 299

In [312]:
many_communities_tfhub_test_df = test_dataset.show_data()

In [313]:
print_results(many_communities_tfhub_test_df, MODEL_NAMES)

tf_hub_classifier_many_communities:v20190219_185602_316000_1553563221
	ROC AUC: 0.5206237424547283
	PR AUC: 0.0060501991156426144
